# การสร้างด้วยโมเดลตระกูล Meta

## บทนำ

บทเรียนนี้จะพูดถึง:

- การสำรวจโมเดลหลักสองตัวในตระกูล Meta - Llama 3.1 และ Llama 3.2
- ทำความเข้าใจกรณีการใช้งานและสถานการณ์ที่เหมาะสมของแต่ละโมเดล
- ตัวอย่างโค้ดเพื่อแสดงจุดเด่นของแต่ละโมเดล

## ตระกูลโมเดล Meta

ในบทเรียนนี้ เราจะสำรวจ 2 โมเดลจากตระกูล Meta หรือที่เรียกว่า "Llama Herd" ได้แก่ Llama 3.1 และ Llama 3.2

โมเดลเหล่านี้มีหลายเวอร์ชัน และสามารถใช้งานได้ในตลาดโมเดลของ Github ดูรายละเอียดเพิ่มเติมเกี่ยวกับการใช้ Github Models เพื่อ [สร้างต้นแบบด้วยโมเดล AI](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)

เวอร์ชันของโมเดล:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*หมายเหตุ: Llama 3 ก็มีให้ใช้งานบน Github Models เช่นกัน แต่จะไม่กล่าวถึงในบทเรียนนี้*


## Llama 3.1

ด้วยจำนวนพารามิเตอร์ 405 พันล้าน Llama 3.1 จัดอยู่ในหมวดหมู่ LLM แบบโอเพ่นซอร์ส

เวอร์ชันนี้เป็นการอัปเกรดจาก Llama 3 รุ่นก่อนหน้า โดยมีจุดเด่นดังนี้:

- หน้าต่างบริบทที่ใหญ่ขึ้น - 128k โทเคน เทียบกับ 8k โทเคน
- จำนวนโทเคนเอาต์พุตสูงสุดที่มากขึ้น - 4096 เทียบกับ 2048
- รองรับหลายภาษาดีขึ้น - เนื่องจากจำนวนโทเคนที่ใช้ฝึกสอนเพิ่มขึ้น

สิ่งเหล่านี้ทำให้ Llama 3.1 สามารถรองรับกรณีการใช้งานที่ซับซ้อนมากขึ้นในการสร้างแอปพลิเคชัน GenAI เช่น:
- การเรียกใช้ฟังก์ชันโดยตรง - สามารถเรียกใช้เครื่องมือหรือฟังก์ชันภายนอกนอกเหนือจากเวิร์กโฟลว์ของ LLM ได้
- ประสิทธิภาพ RAG ที่ดีขึ้น - เนื่องจากหน้าต่างบริบทที่กว้างขึ้น
- การสร้างข้อมูลสังเคราะห์ - สามารถสร้างข้อมูลที่มีประสิทธิภาพสำหรับงานอย่างการปรับแต่งโมเดล


### การเรียกใช้ฟังก์ชันโดยตรง

Llama 3.1 ได้รับการปรับแต่งให้มีประสิทธิภาพมากขึ้นในการเรียกใช้ฟังก์ชันหรือเครื่องมือต่าง ๆ นอกจากนี้ยังมีเครื่องมือในตัวสองอย่างที่โมเดลสามารถระบุได้ว่าควรใช้ตามคำสั่งที่ผู้ใช้ป้อนเข้ามา เครื่องมือเหล่านี้ได้แก่

- **Brave Search** - ใช้ค้นหาข้อมูลล่าสุด เช่น สภาพอากาศ โดยการค้นหาผ่านเว็บ
- **Wolfram Alpha** - ใช้สำหรับคำนวณทางคณิตศาสตร์ที่ซับซ้อนมากขึ้น จึงไม่จำเป็นต้องเขียนฟังก์ชันเอง

คุณยังสามารถสร้างเครื่องมือแบบกำหนดเองที่ LLM สามารถเรียกใช้งานได้ด้วย

ในตัวอย่างโค้ดด้านล่างนี้:

- เรากำหนดเครื่องมือที่มีให้ใช้ (brave_search, wolfram_alpha) ใน system prompt
- ส่งคำถามจากผู้ใช้ที่สอบถามเกี่ยวกับสภาพอากาศในเมืองหนึ่ง
- LLM จะตอบกลับด้วยการเรียกใช้เครื่องมือ Brave Search ซึ่งจะมีลักษณะดังนี้ `<|python_tag|>brave_search.call(query="Stockholm weather")`

*หมายเหตุ: ตัวอย่างนี้เป็นเพียงการเรียกใช้เครื่องมือเท่านั้น หากต้องการผลลัพธ์ คุณจะต้องสมัครบัญชีฟรีที่หน้า Brave API และกำหนดฟังก์ชันเอง*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

แม้ว่า Llama 3.1 จะเป็น LLM แต่ก็ยังมีข้อจำกัดในเรื่องมัลติโมดัล นั่นคือ ความสามารถในการใช้ข้อมูลเข้าแบบต่าง ๆ เช่น รูปภาพเป็นพรอมต์และให้คำตอบกลับ ความสามารถนี้เป็นหนึ่งในฟีเจอร์หลักของ Llama 3.2 โดยฟีเจอร์เหล่านี้ยังรวมถึง:

- มัลติโมดัล - สามารถประเมินทั้งพรอมต์ข้อความและรูปภาพได้
- ขนาดเล็กถึงกลาง (11B และ 90B) - ช่วยให้สามารถปรับใช้งานได้อย่างยืดหยุ่น
- เวอร์ชันเฉพาะข้อความ (1B และ 3B) - ช่วยให้โมเดลสามารถนำไปใช้กับอุปกรณ์ edge หรือมือถือได้ และตอบสนองได้รวดเร็ว

การรองรับมัลติโมดัลถือเป็นก้าวสำคัญในโลกของโมเดลโอเพ่นซอร์ส ตัวอย่างโค้ดด้านล่างนี้จะรับทั้งรูปภาพและพรอมต์ข้อความเพื่อวิเคราะห์รูปภาพด้วย Llama 3.2 90B

### การรองรับมัลติโมดัลด้วย Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## การเรียนรู้ไม่ได้หยุดแค่นี้ เดินหน้าต่อไป

หลังจากจบบทเรียนนี้แล้ว ลองเข้าไปดู [ชุดการเรียนรู้ Generative AI](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) ของเรา เพื่อพัฒนาความรู้ด้าน Generative AI ให้มากยิ่งขึ้น!



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารฉบับนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามอย่างเต็มที่เพื่อความถูกต้อง แต่โปรดทราบว่าการแปลโดยระบบอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่มีความสำคัญ แนะนำให้ใช้บริการแปลโดยนักแปลมืออาชีพ ทางเราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่คลาดเคลื่อนซึ่งเกิดจากการใช้การแปลนี้
